From Operations Research: Models and Methods by Jensen & Bard*

Ten jobs are to be completed by three workers during the next week. Each worker has a 40-hour work week. The times for the workers to complete the jobs are shown in the table. The values in the cells assume that each job is completed by a single worker; however, jobs can be shared, with completion times being determined proportionally If no entry exists in a particular cell, it means that the corresponding job cannot be performed by the corresponding worker. Set up and solve an LP model that will determine the optimal assignment of workers to jobs. The goal is to minimize the total time required to complete all the jobs.

| Workers \ Tasks |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  8 |  9 | 10 |
|:---------------:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|
| A               |  - |  7 |  3 |  - |  - | 18 | 13 |  6 |  - |  9 |
| B               | 12 |  5 |  - | 12 |  4 | 22 |  - | 17 | 13 |  - |
| C               | 18 |  - |  6 |  8 | 10 |  - | 19 |  - |  8 | 15 |

## Model
Define $W$ as the set of workers and $T$ as the sets of tasks. Also, define $c_{wt}$ as the number of hours worker $w$ requires to complete task $t$. (Note that we do not explicitly prohibit a worker from completiting as task; rather, we make the cost arbitrarily large if worker $w$ is unable to perform task $t$.) Let $x_{wt}$ be the proportion of task $t$ that is completed by worker $j$. Let $H$ be the max number of hours that any single worker may log in a week. We formulate as follows.


$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{w \in W} \sum_{t \in T} c_{wt} x_{wt} && \\
\text{subject to  }
& \sum_{t \in T} c_{wt} x_{wt} \le H,
&& \qquad \forall w \in W \\
& \sum_{w \in W} x_{wt} = 1
&& \qquad \forall t \in T \\
& 0 \le x_{wt} \le 1,
&& \qquad \forall w \in W, \forall t \in T
\end{alignat*}
$$

 ### Google Colab

In [ ]:
!pip install -q pyomo
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
ipopt_executable = '/content/ipopt'

     |████████████████████████████████| 9.2 MB 5.0 MB/s 
     |████████████████████████████████| 49 kB 853 kB/s 


## Mac-OS

In [ ]:
!pip install -q pyomo
!curl -s https://ampl.com/dl/open/ipopt/ipopt-osx.zip --output ipopt-osx.zip
!tar xf ipopt-osx.zip ipopt
ipopt_executable = "./ipopt"
!rm ipopt-osx.zip

## Window PC

In [ ]:
!conda install -c conda-forge pyomo pyomo.extras
!conda install -c conda-forge/label/cf201901 ipopt 

In [20]:
from pyomo.environ import *

import matplotlib.pyplot as plt
import random 
import pandas as pd

In [47]:
workers = {'A', 'B', 'C'}

tasks = set(range(1, 11))

c = {
    ('A',  2):  7,
    ('A',  3):  3,
    ('A',  6): 18,
    ('A',  7): 13,
    ('A',  8):  6,
    ('A', 10):  9,
    ('B',  1): 12,
    ('B',  2):  5,
    ('B',  4): 12,
    ('B',  5):  4,
    ('B',  6): 22,
    ('B',  8): 17,
    ('B',  9): 13,
    ('C',  1): 18,
    ('C',  3):  6,
    ('C',  4):  8,
    ('C',  5): 10,
    ('C',  7): 19,
    ('C',  9):  8,
    ('C', 10): 15,
}

max_hours = 40

In [48]:
model = ConcreteModel()
#model = AbstractModel()

model.workers = Set(initialize=workers)
model.tasks =   Set(initialize=tasks)

    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo


In [49]:
model.c         = Param(model.workers, model.tasks, initialize=c, default=1000)
model.max_hours = Param(initialize=max_hours)

In [50]:
model.x = Var(model.workers, model.tasks, domain=Reals, bounds=(0, 1))
#model.x = Var(model.workers, model.tasks, domain=Binary, bounds=(0, 1))



In [51]:
expr = sum(model.c[w, t] * model.x[w, t] for w in model.workers for t in model.tasks)


model.objective = Objective(sense=minimize, expr=expr)


In [52]:
model.tasks_done = ConstraintList()

for t in model.tasks:
    lhs = sum(model.x[w, t] for w in model.workers)
    rhs = 1
    model.tasks_done.add(lhs == rhs)

In [53]:
model.hour_limit = ConstraintList()

for w in model.workers:
    lhs = sum(model.c[w, t] * model.x[w, t] for t in model.tasks)
    rhs = model.max_hours
    model.hour_limit.add(lhs <= rhs)

In [54]:
opt = SolverFactory('ipopt')
results = opt.solve(model, tee=True)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       30
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       30
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       30
                     variables with only upper bounds:        0
Tot

In [55]:
df = pd.DataFrame(index=pd.MultiIndex.from_tuples(model.x, names=['w', 't']))
df['x'] = [value(model.x[key]) for key in df.index]
df['c'] = [model.c[key] for key in df.index]


In [56]:
(df['c'] * df['x']).unstack('t').round()

t,1,2,3,4,5,6,7,8,9,10
w,,,,,,,,,,
A,0.0,0.0,3.0,0.0,0.0,9.0,13.0,6.0,0.0,9.0
B,12.0,5.0,0.0,0.0,4.0,11.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,8.0,0.0


In [57]:
(df['c'] * df['x']).groupby('w').sum().to_frame()

,0
w,
A,40.000040
B,31.999951
C,16.000001


In [58]:
df['x'].groupby('t').sum().to_frame().T

t,1,2,3,4,5,6,7,8,9,10
x,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
